# Previsão de preços com Tamanho, Localização e Vizinhança

Nesse projeto final, tentaremos usar todos os recursos do nosso conjunto de dados para aprimorar o modelo. Isso significa que precisaremos fazer uma limpeza mais cuidadosa do conjunto de dados e considerar alguns dos detalhes mais sutis dos modelos lineares.

In [2]:
!pip install category_encoders --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.1 MB/s eta 0:00:00


In [3]:
import warnings
from glob import glob

import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)

### Preparação dos dados

Importação

In [21]:
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Get place name
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)

    # Drop columns with high null counts
    df.drop(columns=["floor", "expenses"], inplace=True)

    # Drop low- an high-cardinality categorical variables
    df.drop(columns=["operation", "property_type", "currency", "properati_url"], inplace=True)

    # Drop leaky columns
    df.drop(columns=[
        'price',
        'price_aprox_local_currency',
        'price_per_m2',
        'price_usd_per_m2'
        ],
        inplace=True
    )

    return df

In [6]:
files = glob("/content/buenos-aires-real-estate-*.csv")
files

['/content/buenos-aires-real-estate-4.csv',
 '/content/buenos-aires-real-estate-2.csv',
 '/content/buenos-aires-real-estate-5.csv',
 '/content/buenos-aires-real-estate-1.csv',
 '/content/buenos-aires-real-estate-3.csv']

In [22]:
frames = [wrangle(file) for file in files]
print("Total de data frames: ", len(frames))
for frame in frames:
    print(frame.shape)

Total de data frames:  5
(1305, 7)
(1315, 7)
(1331, 7)
(1343, 7)
(1288, 7)


In [23]:
df = pd.concat(frames, ignore_index=True)
print(df.shape)
df.head()

(6582, 7)


,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,lat,lon,neighborhood
0,59000.0,0.0,40.0,2.0,-34.604069,-58.460209,Paternal
1,78900.0,43.0,38.0,1.0,-34.620026,-58.388467,Congreso
2,240000.0,97.0,88.0,4.0,-34.599857,-58.489002,Agronomía
3,75000.0,NaN,34.0,2.0,-34.612338,-58.401328,Balvanera
4,110000.0,NaN,46.0,2.0,-34.586160,-58.475633,Parque Chas


### Exploração

* A primeira coisa que precisamos considerar ao tentar usar todos os recursos do df são os valores ausentes. Embora seja possível imputar valores ausentes, ainda é necessário que haja dados suficientes em uma coluna para uma boa imputação. Uma regra geral é que, se mais da metade dos dados em uma coluna estiverem ausentes, é melhor descartá-los do que tentar imputar

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582 entries, 0 to 6581
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   6582 non-null   object 
 1   property_type               6582 non-null   object 
 2   price                       6582 non-null   float64
 3   currency                    6582 non-null   object 
 4   price_aprox_local_currency  6582 non-null   float64
 5   price_aprox_usd             6582 non-null   float64
 6   surface_total_in_m2         4752 non-null   float64
 7   surface_covered_in_m2       6582 non-null   float64
 8   price_usd_per_m2            4536 non-null   float64
 9   price_per_m2                6582 non-null   float64
 10  floor                       1900 non-null   float64
 11  rooms                       5286 non-null   float64
 12  expenses                    1739 non-null   float64
 13  properati_url               6582 

As colunas "floor" e "expenses" possuem uma alta quantidade de valores nulos, mais de 50%, o que faz sentido a exclusão dessas colunas.

Ajustei a noss função wrangle() realizar a exclusão das colunas mencionadas.

* A próxima coisa que precisamos observar são colunas categóricas com cardinalidade baixa ou alta. Se houver apenas uma categoria em uma coluna, ela não fornecerá nenhuma informação exclusiva para o nosso modelo. No outro extremo, colunas em que quase todas as linhas têm sua própria categoria não ajudarão nosso modelo a identificar tendências úteis nos dados.

Vamos dar uma olhada na cardinalidade dos nossos recursos.

In [15]:
df.select_dtypes("object").head()

,operation,property_type,currency,properati_url,neighborhood
0,sell,apartment,USD,http://paternal.properati.com.ar/12t3j_venta_d...,Paternal
1,sell,apartment,USD,http://congreso.properati.com.ar/12j7y_venta_d...,Congreso
2,sell,apartment,USD,http://agronomia.properati.com.ar/10vec_venta_...,Agronomía
3,sell,apartment,USD,http://balvanera.properati.com.ar/10vnl_venta_...,Balvanera
4,sell,apartment,USD,http://parque-chas.properati.com.ar/yc23_venta...,Parque Chas


In [16]:
df.select_dtypes("object").nunique()

,0
operation,1
property_type,1
currency,2
properati_url,6582
neighborhood,57


Podemos observar que as colunas "operation", "property_type" e "currency" possuem baixa cardinalidade, ou seja, apresentam poucos valores distintos e, consequentemente, contribuem com pouca informação útil para os nossos modelos.

Por outro lado, a coluna "properati_url" possui alta cardinalidade, apresentando praticamente um valor único para cada observação do DataFrame. Isso também não agrega valor relevante à modelagem, pois não há nenhuma tendência ou padrão que possa ser identificado. Além disso, caso utilizássemos o OneHotEncoder, seriam geradas 6.582 novas colunas, o que tornaria a iteração do modelo extremamente ineficiente.

Ajustei a nossa função wrangle() para realizar a exclusão das colunas mencionadas acima.

* Também é importante que descartemos quaisquer colunas que possam constituir vazamento, ou seja, recursos que foram criados usando nosso destino ou que forneceriam ao nosso modelo informações às quais ele não teria acesso quando fosse implantado.

In [20]:
sorted(df.columns)

['lat',
 'lon',
 'neighborhood',
 'price',
 'price_aprox_local_currency',
 'price_aprox_usd',
 'price_per_m2',
 'price_usd_per_m2',
 'rooms',
 'surface_covered_in_m2',
 'surface_total_in_m2']

A nossa variável alvo é "price_aprox_usd", porém o DataFrame contém outras variáveis relacionadas a preço que podem fornecer pistas ao modelo, levando-o a “roubar” durante as previsões. Isso acontece porque queremos que o modelo aprenda a prever o preço sem ter acesso direto a informações que já antecipam o resultado.

É como se estivéssemos tentando prever o placar de um jogo de futebol, mas o modelo soubesse previamente quantos pontos o time conquistou ao final da partida. Com essa informação, fica fácil saber se o time venceu ou perdeu, o que invalida o processo de previsão real.

Ajuestei nossa função wrangle() para excluir as colunas relacionadas a preço.